In [1]:
from dotenv import load_dotenv
import json
import os
import tweepy

In [2]:
load_dotenv()

True

In [3]:
bearer_token = os.environ.get('bearer_token')
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

In [4]:
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
)

In [5]:
# Get Twitter ID from handle from https://tweeterid.com/
accounts = [
    {
        'handle': 'cnnbrk',
        'id': 428333
    },
    {
        'handle': 'CNN',
        'id': 759251
    },
    {
        'handle': 'nytimes',
        'id': 807095
    },
    {
        'handle': 'BBCBreaking',
        'id': 5402612
    },
    {
        'handle': 'BBCWorld',
        'id': 742143
    },
    {
        'handle': 'TheEconomist',
        'id': 5988062
    },
    {
        'handle': 'WSJ',
        'id': 3108351
    },
    {
        'handle': 'washingtonpost',
        'id': 2467791
    },
    {
        'handle': 'TIME',
        'id': 14293310
    },
    {
        'handle': 'ABC',
        'id': 28785486
    },
    {
        'handle': 'ndtv',
        'id': 37034483
    },
    {
        'handle': 'AP',
        'id': 51241574
    },
    {
        'handle': 'XHNews',
        'id': 487118986
    },
    {
        'handle': 'HuffPost',
        'id': 14511951
    },
    {
        'handle': 'guardian',
        'id': 87818409
    },
    {
        'handle': 'BreakingNews',
        'id': 6017542
    },
    {
        'handle': 'SkyNews',
        'id': 7587032
    },
    {
        'handle': 'AJEnglish',
        'id': 4970411
    },
    {
        'handle': 'FT',
        'id': 18949452
    },
    {
        'handle': 'SkyNewsBreak',
        'id': 87416722
    },
    {
        'handle': 'politico',
        'id': 9300262
    },
    {
        'handle': 'CNBC',
        'id': 20402945
    },
    {
        'handle': 'FRANCE24',
        'id': 1994321
    },
    {
        'handle': 'guardiannews',
        'id': 788524
    },
    {
        'handle': 'Independent',
        'id': 16973333
    },
    {
        'handle': 'BBCAfrica',
        'id': 36670025
    },
    {
        'handle': 'Newsweek',
        'id': 2884771
    },
    {
        'handle': 'Telegraph',
        'id': 16343974
    },
    {
        'handle': 'RT_com',
        'id': 64643056
    },
    {
        'handle': 'CBCNews',
        'id': 6433472
    },
    {
        'handle': 'FinancialTimes',
        'id': 4898091
    },
    {
        'handle': 'Reuters',
        'id': 1652541
    },
    {
        'handle': 'SportsCenter',
        'id': 26257166
    },
    {
        'handle': 'espn',
        'id': 2557521
    }
]

In [6]:
def get_tweets(accounts, n):
    tweets = []
    
    for account in accounts:
        response = client.get_users_tweets(account['id'], max_results=n)
        
        for tweet in response.data:
            tweets.append(tweet.text)
    
    return tweets

In [7]:
get_tweets(accounts, 10)

['Man accused of attacking Paul Pelosi, husband of House Speaker Nancy Pelosi, pleads not guilty to all state charges during initial court appearance\nhttps://t.co/YJXVfIx0BI',
 'Former Israeli leader Benjamin Netanyahu may be on the verge of a comeback, as exit polls suggest his bloc could win a majority in parliament\nhttps://t.co/5SBcCDvaxQ',
 'Brazilian President Jair Bolsonaro does not concede election loss in speech after days of silence, but signals cooperation with transfer of power\nhttps://t.co/4FStE9Qrg5',
 'Two police officers were shot Tuesday afternoon in Newark, New Jersey, and authorities are looking for a suspect, according to several law enforcement sources https://t.co/Bq46PWunMb',
 "Supreme Court declines to block subpoena for Sen. Lindsey Graham's testimony in front of Atlanta grand jury probing efforts to overturn 2020 election\nhttps://t.co/h91Wf797FL",
 'Takeoff, who was one third of the platinum-selling rap group Migos, was shot and killed early Tuesday in Hous